## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
# 模仿 STRIVING FOR SIMPLICITY: THE ALL CONVOLUTIONAL NET 
# (Author: Jost Tobias Springenberg, Alexey Dosovitskiy, Thomas Brox, Martin Riedmiller)
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

batch_size = 256
num_classes = 10
epochs = 10

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

data_gen = ImageDataGenerator(rotation_range=20,width_shift_range=0.2,height_shift_range=0.2,
                              featurewise_std_normalization=True,horizontal_flip=True)
data_gen.fit(x_train)


# pooling 用 conv2d strides=2代替
# 只做兩組，電腦無GPU

model = Sequential()
model.add(Conv2D(96, (3, 3), padding='same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(96, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(96, (3, 3), strides=2))

model.add(Conv2D(192, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(192, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(192, (3, 3), strides=2))

model.add(Flatten())
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()




x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:349: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 96)        2688      
_________________________________________________________________
activation (Activation)      (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 96)        83040     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 96)        83040     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 192)       166080    
_________________________________________________________________
activation_2 (Activation)    (None, 15, 15, 192)       0

In [2]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])

history = model.fit(data_gen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train) // batch_size,
                    epochs=epochs,verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
195/195 [==============================] - 592s 3s/step - loss: 1.7007 - accuracy: 0.3747 - val_loss: 64.6462 - val_accuracy: 0.2847
Epoch 2/10
195/195 [==============================] - 596s 3s/step - loss: 1.3413 - accuracy: 0.5160 - val_loss: 66.5711 - val_accuracy: 0.2856
Epoch 3/10
195/195 [==============================] - 596s 3s/step - loss: 1.1611 - accuracy: 0.5865 - val_loss: 74.6497 - val_accuracy: 0.2892
Epoch 4/10
195/195 [==============================] - 594s 3s/step - loss: 1.0243 - accuracy: 0.6389 - val_loss: 68.8279 - val_accuracy: 0.3105
Epoch 5/10
195/195 [==============================] - 604s 3s/step - loss: 0.9320 - accuracy: 0.6759 - val_loss: 63.5670 - val_accuracy: 0.3600
Epoch 6/10
195/195 [==============================] - 601s 3s/step - loss: 0.8575 - accuracy: 0.7016 - val_loss: 66.3888 - val_accuracy: 0.3716
Epoch 7/10
195/195 [==============================] - 603s 3s/step - loss: 0.8037 - accuracy: 0.7208 - val_loss: 60.6344 - val_accuracy: